# Experiments
1. NN Classification
2. NN Regression

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import io
import os
import json
import time
import sys
import math
import copy
import pickle
import zipfile
from textwrap import wrap
from pathlib import Path
from itertools import zip_longest
from collections import defaultdict
from urllib.error import URLError
from urllib.request import urlopen

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch import tensor
from torch.nn import functional as F 
from torch.optim.lr_scheduler import _LRScheduler

In [6]:
sys.path.append("/Users/varunn/Documents/ExternalTest/MAD/src/")
from constants import *
from metadata_utils import _find_files
from baseline_feats_utils import feat_type_feats_dct

In [7]:
feat_type_feats_dct

{'user': ['num_interactions',
  'mean_price_interactions',
  'earliest_interaction_date',
  'min_num_interactions_per_pdt',
  'max_num_interactions_per_pdt',
  'mean_num_interactions_per_pdt',
  'min_num_interactions_per_ont',
  'max_num_interactions_per_ont',
  'mean_num_interactions_per_ont',
  'min_num_interactions_per_brand',
  'max_num_interactions_per_brand',
  'mean_num_interactions_per_brand'],
 'item': ['num_interactions',
  'earliest_interaction_date',
  'min_num_interactions_per_user',
  'max_num_interactions_per_user',
  'mean_num_interactions_per_user']}

## Experiment 1 - NN Classification with baseline features

In [20]:
from torch.utils.data import IterableDataset
from itertools import chain, islice


class InteractionsStream(IterableDataset):

    def __init__(self, sample, model_type, file_name=None,
                 interim_data_dir=INTERIM_DATA_DIR, user_col=USER_COL,
                 item_col=ITEM_COL, ontology_col=ONTOLOGY_COL,
                 brand_col=BRAND_COL, price_col=PRICE_COL, dv_col=DV_COL,
                 date_col=DATE_COL, end_token='.gz', chunksize=10):

        data_dir = interim_data_dir
        
        if file_name is None:
            files = _find_files(data_dir, end_token)
            if sample == 'train':
                self.files = [os.path.join(data_dir, x) for x in files
                              if not x.startswith('0005')]
            elif sample == 'test':
                self.files = [os.path.join(data_dir, x) for x in files
                              if x.startswith('0005')]
        else:
            self.files = [os.path.join(data_dir, file_name)]
        print(self.files)
        
        self.model_type = model_type
        self.user_col = user_col
        self.item_col = item_col
        self.ontology_col = ontology_col
        self.brand_col = brand_col
        self.price_col = price_col
        self.date_col = date_col
        self.dv_col = dv_col
        self.feat_type_feats_dct = feat_type_feats_dct
        self.chunksize = chunksize
        user_feats = ['{}_{}'.format(self.user_col, x) for x in
                      self.feat_type_feats_dct['user']
                      if x != 'earliest_interaction_date']
        user_feats.append('{}_days_since_earliest_interaction'.format(
            self.user_col))
        item_feats = ['{}_{}'.format(self.item_col, x) for x in
                      self.feat_type_feats_dct['item']
                      if x != 'earliest_interaction_date']
        item_feats.append('{}_days_since_earliest_interaction'.format(
            self.item_col))
        self.numeric_feats = [self.price_col] + user_feats + item_feats
        self.cat_feats = [self.user_col, self.item_col, self.ontology_col,
                          self.brand_col]
        

    def read_file(self, fn):
        
        df = pd.read_csv(fn, compression='gzip', sep='|', iterator=True,
                         chunksize=self.chunksize)
        return df
    
    def get_dv_for_classification(self, dv_lst):
        
        if self.model_type == 'classification':
            return [int(x-1) for x in dv_lst]
        else:
            return [int(x) for x in dv_lst]

    def process_data(self, fn):

        print('read data')
        data = self.read_file(fn)

        for row in data:
            x1 = row[self.cat_feats].values.tolist()
            x2 = row[self.numeric_feats].values.tolist()
            y = self.get_dv_for_classification(
                    row[self.dv_col].tolist())
            yield (x1, x2, y)

    def get_stream(self, files):
        return chain.from_iterable(map(self.process_data, files))

    def __iter__(self):
        return self.get_stream(self.files)

In [21]:
class ProductRecommendationModel(nn.Module):
    """
    Defines the neural network for product recommendation
    """

    def __init__(self, embedding_sizes, n_cont, n_classes=3):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for
                                         categories, size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings)
        self.n_emb, self.n_cont, self.n_classes = n_emb, n_cont, n_classes
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 300)
        self.lin2 = nn.Linear(300, 100)
        self.lin3 = nn.Linear(100, self.n_classes)
        self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(300)
        self.bn3 = nn.BatchNorm1d(100)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)


    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)

        return x

In [57]:
import torch.optim as torch_optim
from torch import tensor
from sklearn.metrics import roc_auc_score
from tqdm import tqdm


def choose_embedding_size(cat_cols, cat_num_values, min_emb_dim=100):
    """
    cat_cols: list of categorical columns
    cat_num_values: list of number of unique values for each categorical column
    """

    embedded_cols = dict(zip(cat_cols, cat_num_values))
    embedding_sizes = [(n_categories, min(min_emb_dim, (n_categories+1)//2))
                       for _, n_categories in embedded_cols.items()]
    return embedding_sizes


def get_default_device():
    """Pick GPU if available, else CPU"""

    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""

    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


def get_optimizer(model, lr = 0.001, wd = 0.0):

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim


def construct_tensor(a):

    final = []
    for i in a:
        out = []
        for j in i:
            out.append(j.tolist())
        out1 = []
        for item in zip(*out):
            out1.append(list(item))
        final += out1
    return tensor(final)


def construct_tensor_y(a):

    out = []
    for i in a:
        out += i.tolist()
    return tensor(out)


def train_model(model, optim, train_dl, train_size, chunksize, batch_size,
                device, loss_fn=F.cross_entropy):

    model.train()
    total = 0
    sum_loss = 0
    with tqdm(total=train_size // (batch_size * chunksize)) as pbar:
        for x1, x2, y in train_dl:
            x1, x2, y = (construct_tensor(x1), construct_tensor(x2),
                         construct_tensor_y(y))
            x1 = x1.to(device)
            x2 = x2.to(device)
            y = y.to(device)
            batch = y.size()[0]
            output = model(x1, x2)
            loss = loss_fn(output, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            total += batch
            sum_loss += loss.item()
            pbar.update(1)
    return sum_loss/total


def val_loss(model, valid_dl, test_size, chunksize, batch_size,
             device, loss_fn=F.cross_entropy):

    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    sum_auc_macro = 0
    sum_auc_micro = 0
    num_aucs = 0
    with tqdm(total=test_size // (batch_size * chunksize)) as pbar:
        for x1, x2, y in valid_dl:
            x1, x2, y = (construct_tensor(x1), construct_tensor(x2),
                         construct_tensor_y(y))
            x1 = x1.to(device)
            x2 = x2.to(device)
            y = y.to(device)
            batch = y.size()[0]
            out = model(x1, x2)
            loss = loss_fn(out, y)
            sum_loss += loss.item()
            total += batch
            pred = torch.max(out, 1)[1]
            pred_prob = F.softmax(out, dim=1)
            y_onehot = F.one_hot(y)
            correct += (pred == y).float().sum().item()
            pred_prob = pred_prob.cpu().detach().numpy()
            y_onehot = y_onehot.cpu().detach().numpy()
            try:
                sum_auc_macro += roc_auc_score(y_onehot, pred_prob,
                                               average='macro')
                sum_auc_micro += roc_auc_score(y_onehot, pred_prob,
                                               average='micro')
                num_aucs += 1
            except:
                continue

            pbar.update(1)
    print("valid loss %.3f, accuracy %.3f, macro auc %.3f and micro auc %.3f" % (
        sum_loss/total, correct/total, sum_auc_macro/num_aucs, sum_auc_micro/num_aucs))
    return sum_loss/total, correct/total, sum_auc_macro/num_aucs, sum_auc_micro/num_aucs


def train_loop(model, train_dl, valid_dl, epochs, train_size,
               test_size, chunksize, batch_size, device, lr=0.01,
               wd=0.0, loss_fn=F.cross_entropy):

    optim = get_optimizer(model, lr = lr, wd = wd)
    start = time.time()
    losses = []
    for i in range(epochs):
        stats = {'epoch': i+1}
        train_loss = train_model(model, optim, train_dl, train_size,
                                 chunksize, batch_size, device,
                                 loss_fn)
        print("training loss: ", train_loss)
        stats['train_loss'] = train_loss
        loss, acc, auc_macro, auc_micro = val_loss(
            model, valid_dl, test_size, chunksize, batch_size, device, loss_fn)
        print('time taken: %0.2f' % (time.time() - start))
        stats['test_loss'] = loss
        stats['test_acc'] = acc
        stats['test_auc_macro'] = auc_macro
        stats['test_auc_micro'] = auc_micro
        losses.append(stats)
    return losses

In [33]:
# GLOBALS
TRAIN_FILE_NAME = '0000_part_00.gz'
TEST_FILE_NAME = '0005_part_07.gz'
N_USERS = 10130223
N_ITEMS = 1175648
N_ONTOLOGIES = 801
N_BRANDS = 1686
BATCH_SIZE = 50
CHUNKSIZE = 100
TRAIN_SIZE = 4812995 # corresponds to FILE_NAME
TEST_SIZE = 1371989    # corresponds to FILE_NAME

In [34]:
# choose embedding size

cat_cols = [USER_COL, ITEM_COL, ONTOLOGY_COL, BRAND_COL]
cat_num_values = [N_USERS, N_ITEMS, N_ONTOLOGIES, N_BRANDS]
embedding_sizes = choose_embedding_size(cat_cols, cat_num_values, 150)

In [35]:
embedding_sizes

[(10130223, 150), (1175648, 150), (801, 150), (1686, 150)]

In [50]:
# dataset

from torch.utils.data import DataLoader

train_dataset = InteractionsStream(
    file_name=TRAIN_FILE_NAME, model_type='classification',
    sample='train', chunksize=CHUNKSIZE)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          shuffle=False)

test_dataset = InteractionsStream(
    file_name=TEST_FILE_NAME, model_type='classification',
    sample='test', chunksize=CHUNKSIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False)

['/Users/varunn/Documents/ExternalTest_Data/MAD/interim/0000_part_00.gz']
['/Users/varunn/Documents/ExternalTest_Data/MAD/interim/0005_part_07.gz']


In [51]:
device = get_default_device()
device

device(type='cpu')

In [52]:
n_cont = len(train_loader.dataset.numeric_feats)
print('number of numeric vars: ', n_cont)

net = ProductRecommendationModel(embedding_sizes, n_cont, 3)

number of numeric vars:  18


In [53]:
net

ProductRecommendationModel(
  (embeddings): ModuleList(
    (0): Embedding(10130223, 150)
    (1): Embedding(1175648, 150)
    (2): Embedding(801, 150)
    (3): Embedding(1686, 150)
  )
  (lin1): Linear(in_features=618, out_features=300, bias=True)
  (lin2): Linear(in_features=300, out_features=100, bias=True)
  (lin3): Linear(in_features=100, out_features=3, bias=True)
  (bn1): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (drops): Dropout(p=0.3, inplace=False)
)

In [54]:
to_device(net, device)

ProductRecommendationModel(
  (embeddings): ModuleList(
    (0): Embedding(10130223, 150)
    (1): Embedding(1175648, 150)
    (2): Embedding(801, 150)
    (3): Embedding(1686, 150)
  )
  (lin1): Linear(in_features=618, out_features=300, bias=True)
  (lin2): Linear(in_features=300, out_features=100, bias=True)
  (lin3): Linear(in_features=100, out_features=3, bias=True)
  (bn1): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (drops): Dropout(p=0.3, inplace=False)
)

In [55]:
from itertools import islice

for x1, x2, y in islice(train_loader, 2):
    x1, x2, y = (construct_tensor(x1), construct_tensor(x2),
                 construct_tensor_y(y))
    print(x1)
    print('\n')
    print(x2)
    print('\n')
    print(y)
    print('\n\n\n')

read data
tensor([[ 6301963,  1084662,      431,     1356],
        [ 5588197,   674750,      783,     1147],
        [ 7712525,   886279,      512,     1592],
        ...,
        [ 3451614,   787244,      431,      616],
        [ 2818598,   303725,      104,      279],
        [10071369,   655860,      277,     1289]])


tensor([[8.9900e+02, 9.2000e+01, 1.7840e+03,  ..., 6.0000e+00, 1.0883e+00,
         4.0625e-03],
        [1.2990e+03, 2.0000e+00, 1.0490e+03,  ..., 7.0000e+00, 1.0166e+00,
         1.4421e-02],
        [7.9900e+02, 7.0000e+00, 1.1552e+03,  ..., 9.0000e+00, 1.0309e+00,
         3.8264e-02],
        ...,
        [6.9900e+02, 2.0920e+03, 1.4653e+03,  ..., 1.4000e+01, 1.0604e+00,
         4.2753e-01],
        [2.4561e+04, 7.0000e+00, 2.1362e+04,  ..., 5.6000e+01, 1.1150e+00,
         3.6240e-01],
        [1.2990e+03, 1.0000e+01, 1.2334e+03,  ..., 2.3000e+01, 1.2392e+00,
         3.5769e-01]])


tensor([0, 0, 0,  ..., 0, 0, 0])




tensor([[7623743,  532609,     431,    

In [49]:
for x1, x2, y in islice(test_loader, 2):
    x1, x2, y = (construct_tensor(x1), construct_tensor(x2),
                 construct_tensor_y(y))
    x1 = x1.to(device)
    x2 = x2.to(device)
    y = y.to(device)
    print(x1)
    print('\n')
    print(x2)
    print('\n')
    print(y)
    print('\n\n\n')
    print('shape of y: ', y.size())
    print('shape of x1: ', x1.size())
    out = net(x1, x2)
    print('model output: ', out)
    loss = F.cross_entropy(out, y)
    print('Loss: ', loss.item())

read data
tensor([[4852310,  911340,     431,    1480],
        [ 551584,  693320,     696,    1507],
        [6013644,  876085,     217,    1327],
        [2954929,  630337,     329,    1222]])


tensor([[1.0990e+03, 7.7600e+02, 2.6948e+03, 1.0000e+00, 1.3000e+01, 1.0270e+00,
         1.0000e+00, 2.6000e+01, 1.5190e+00, 1.0000e+00, 2.3000e+01, 1.3889e+00,
         8.8655e+00, 1.8800e+02, 1.0000e+00, 4.0000e+00, 1.0271e+00, 3.8233e+01],
        [1.9990e+03, 9.3000e+01, 2.4667e+03, 1.0000e+00, 5.0000e+00, 2.2821e+00,
         1.0000e+00, 8.0000e+00, 2.3345e+00, 1.0000e+00, 5.0000e+00, 2.3013e+00,
         4.4421e-02, 4.7400e+02, 1.0000e+00, 3.0000e+00, 1.0091e+00, 1.5834e+01],
        [5.3990e+03, 1.0150e+03, 2.6539e+03, 1.0000e+00, 1.1000e+01, 1.0242e+00,
         1.0000e+00, 1.7000e+01, 1.2651e+00, 1.0000e+00, 1.9000e+01, 1.1541e+00,
         3.6472e+01, 1.2100e+03, 1.0000e+00, 4.0000e+00, 1.0462e+00, 4.6897e+01],
        [6.9990e+03, 1.4200e+02, 9.2414e+03, 1.0000e+00, 1.3000e+01, 1.

In [ ]:
losses = train_loop(model=net, train_dl=train_loader,
                    valid_dl=test_loader, epochs=1,
                    train_size=TRAIN_SIZE, test_size=TEST_SIZE,
                    chunksize=CHUNKSIZE, batch_size=BATCH_SIZE,
                    device=device, lr=0.02, wd=0.00001,
                    loss_fn=F.cross_entropy)

  0%|          | 0/962 [00:00<?, ?it/s]

read data
